In [ ]:
# using Pkg
# Pkg.add(url = "https://github.com/JuliaReliab/JuliaMBD.git", rev="develop")
using Pkg
Pkg.update()

In [ ]:
# using Pkg
# Pkg.add("DifferentialEquations")
# using Pkg
# Pkg.add("Plots")
using DifferentialEquations
using Plots

In [ ]:
using JuliaMBD
import JuliaMBD: expr, expr_initial, get_inports, get_outports

In [ ]:
module MyModule

using JuliaMBD
import JuliaMBD: expr, expr_initial, get_inports, get_outports

@model MSD begin
    @parameter begin
        M::Float64
        D::Float64
        k::Float64
        g = 9.8
    end
    @block begin
        in1 = InBlock(:in1)
        out1 = OutBlock(:out1)
        constant1 = Constant(:(M*g))
        gain1 = Gain(:D)
        gain2 = Gain(:k)
        gain3 = Gain(:(1/M))
        int1 = Integrator(:s1)
        int2 = Integrator(:s2, initialcondition=:(M*g/k))
        add = Add(inports=[InPort(), InPort(), InPort(), InPort()], signs=[:+, :+, :-, :-])
    end

    in1 => add.inports[1]
    constant1 => add.inports[2]
    gain1 => add.inports[3]
    gain2 => add.inports[4]
    add => gain3 => int1 => [int2, gain1]
    int2 => [gain2, out1]
end

@model TestBlock begin
    @parameter begin
        M::Float64 = 10.0
        D::Float64 = 10.0
        k::Float64 = 10.0
        f::Float64 = 10.0
        p_cycle::Float64 = 20.0
        p_width::Float64 = 50.0
    end
    @block begin
        msd = MSD(out1=OutPort(:x))
        pulse = PulseGenerator(amplitude=:f, period=:p_cycle, pulsewidth=:p_width, phasedelay=10.0, outport=OutPort(:F))
    end

    @scope begin
        pulse
        msd.out1
    end

    pulse => msd.in1
end

end

In [ ]:
?MyModule.TestBlockFunction # check interface of systemfunction

In [ ]:
MyModule.TestBlockFunction(time=1, s1in=0, s2in=0) # check interface of systemfunction

In [ ]:
MyModule.TestBlockInitialFunction(time=1, s1in=0, s2in=0) # check interface of systemfunction

In [ ]:
function ifunc()
    result = MyModule.TestBlockInitialFunction(time=0.0, s1in=0.0, s2in=0.0)
    [result.s1out, result.s2out]
end

function sfunc(du, u, p, t)
    result = MyModule.TestBlockFunction(time=t, s1in=u[1], s2in=u[2])
    (du[1], du[2]) = (result.s1out, result.s2out)
end

In [ ]:
@time begin
    iv = ifunc()
    problem = ODEProblem(sfunc, iv, (0.0, 60.0))
    solution = solve(problem)

    tt = LinRange(0, 60.0, 1000)
    result = [MyModule.TestBlockFunction(time=t, s1in=solution(t)[1], s2in=solution(t)[2]) for t = tt]
    F = [x.scope_F for x = result]
    x = [x.scope_x for x = result]
    plot(tt, [F, x], layout=(2,1), leg=false)
end